## Import all packages and functions needed

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

## Decision tree class

In [68]:
class dtree:
	""" A basic Decision Tree"""

	def __init__(self): # include feature names specific to breast cancer dataset
		""" Constructor """; self.featureNames = ["radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean", "compactness_mean", "concavity_mean", "concave_points_mean", "symmetry_mean", "fractal_dimension_mean", "radius_SE", "texture_SE", "perimeter_SE", "area_SE", "smoothness_SE", "compactness_SE", "concavity_SE", "concave_points_SE", "symmetry_SE", "fractal_dimension_SE", "radius_worst", "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst", "compactness_worst", "concavity_worst", "concave_points_worst", "symmetry_worst", "fractal_dimension_worst"]

	def read_data(self, filename):
		fid = open(filename, "r")
		data = []
		d = []
		for line in fid.readlines():
			d.append(line.strip())
		for d1 in d:
			data.append(d1.split(","))
		fid.close()

		self.classes = []
		for d in range(len(data)): # change classes from strings to integers, remove first column
			self.classes.append(0 if data[d][1] == "M" else 1)
			data[d] = data[d][2:]

		return data, self.classes, self.featureNames

	def classify(self, tree, datapoint): # final class is an integer, changed from string
		if type(tree) == int:
			# Have reached a leaf
			return tree
		else:
			a = list(tree.keys())[0]
			for i in range(len(self.featureNames)):
				if self.featureNames[i] == a:
					break
			try:
				t = tree[a][datapoint[i]]
				return self.classify(t, datapoint)
			except:
				return None

	def classifyAll(self, tree, data):
		results = []
		for i in range(len(data)):
			results.append(self.classify(tree, data[i]))
		return results

	def make_tree(self,
				  data,
				  classes,
				  featureNames,
				  maxlevel=-1,
				  level=0,
				  forest=0):
		""" The main function, which recursively constructs the tree"""

		nData = len(data)
		nFeatures = len(data[0])

		try:
			self.featureNames
		except:
			self.featureNames = featureNames

		# List the possible classes
		newClasses = []
		for aclass in classes:
			if newClasses.count(aclass) == 0:
				newClasses.append(aclass)

		# Compute the default class (and total entropy)
		frequency = np.zeros(len(newClasses))

		totalEntropy = 0
		index = 0
		for aclass in newClasses:
			frequency[index] = classes.count(aclass)
			totalEntropy += self.calc_entropy(float(frequency[index]) / nData)
			index += 1

		default = classes[np.argmax(frequency)]

		if nData == 0 or nFeatures == 0 or (maxlevel >= 0
											and level > maxlevel):
			# Have reached an empty branch
			return default
		elif classes.count(classes[0]) == nData:
			# Only 1 class remains
			return classes[0]
		else:

			# Choose which feature is best
			gain = np.zeros(nFeatures)
			featureSet = list(range(nFeatures))
			if forest != 0:
				np.random.shuffle(featureSet)
				featureSet = featureSet[0:forest]
			for feature in featureSet:
				g = self.calc_info_gain(data, classes, feature)
				gain[feature] = totalEntropy - g

			bestFeature = np.argmax(gain)
			tree = {featureNames[bestFeature]: {}}

			# List the values that bestFeature can take
			values = []
			for datapoint in data:
				if datapoint[feature] not in values:
					values.append(datapoint[bestFeature])

			for value in values:
				# Find the datapoints with each feature value
				newData = []
				newClasses = []
				index = 0
				for datapoint in data:
					if datapoint[bestFeature] == value:
						if bestFeature == 0:
							newdatapoint = datapoint[1:]
							newNames = featureNames[1:]
						elif bestFeature == nFeatures:
							newdatapoint = datapoint[:-1]
							newNames = featureNames[:-1]
						else:
							newdatapoint = datapoint[:bestFeature]
							# newdatapoint.append(datapoint[bestFeature+1:])
							newdatapoint = np.append(
								newdatapoint, datapoint[bestFeature + 1:])
							newNames = featureNames[:bestFeature]
							# newNames.append(featureNames[bestFeature+1:])
							newNames = np.append(
								newNames, featureNames[bestFeature + 1:])
						newData.append(newdatapoint)
						newClasses.append(classes[index])
					index += 1

				# Now recurse to the next level
				subtree = self.make_tree(newData, newClasses, newNames,
										 maxlevel, level + 1, forest)

				# And on returning, add the subtree on to the tree
				tree[featureNames[bestFeature]][value] = subtree

			return tree

	def printTree(self, tree, name):
		if type(tree) == dict:
			print(name, tree.keys()[0])
			for item in tree.values()[0].keys():
				print(name, item)
				self.printTree(tree.values()[0][item], name + "\t")
		else:
			print(name, "\t->\t", tree)

	def calc_entropy(self, p):
		if p != 0:
			return -p * np.log2(p)
		else:
			return 0

	def calc_info_gain(self, data, classes, feature):

		# Calculates the information gain based on entropy
		gain = 0
		nData = len(data)

		# List the values that feature can take

		values = []
		for datapoint in data:
			if datapoint[feature] not in values:
				values.append(datapoint[feature])

		featureCounts = np.zeros(len(values))
		entropy = np.zeros(len(values))
		valueIndex = 0
		# Find where those values appear in data[feature] and the corresponding class
		for value in values:
			dataIndex = 0
			newClasses = []
			for datapoint in data:
				if datapoint[feature] == value:
					featureCounts[valueIndex] += 1
					newClasses.append(classes[dataIndex])
				dataIndex += 1

			# Get the values in newClasses
			classValues = []
			for aclass in newClasses:
				if classValues.count(aclass) == 0:
					classValues.append(aclass)

			classCounts = np.zeros(len(classValues))
			classIndex = 0
			for classValue in classValues:
				for aclass in newClasses:
					if aclass == classValue:
						classCounts[classIndex] += 1
				classIndex += 1

			for classIndex in range(len(classValues)):
				entropy[valueIndex] += self.calc_entropy(
					float(classCounts[classIndex]) / np.sum(classCounts))

			# Computes the entropy
			gain = gain + float(
				featureCounts[valueIndex]) / nData * entropy[valueIndex]
			valueIndex += 1
		return gain

## Testing decision tree

In [156]:
data, classes, names = dtree().read_data('wdbc.data')
X_train, X_test, y_train, y_test = train_test_split(data, classes, test_size=0.5, random_state = 3)
tree = dtree().make_tree(X_test, y_test, names, forest=0)
res = dtree().classifyAll(tree, X_test)
count = 0
for i in range(len(res)):
  if res[i] == y_test[i]:
    count += 1
print("Accuracy:", count / len(res))
#print(1,2,3)

Accuracy: 0.9859649122807017


## Results

The random forest predictive model performs slightly better than the Multi-Layer Perceptron model implemented in Project 2. This is most likely due to bootstrapping in the random forest model, which reduces overfitting and improves the stability of decision trees. The model can be improved by tweaking the number of trees in the model and the number of features used.